In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
!pip install transformers==3.0.2

     |████████████████████████████████| 778kB 7.4MB/s 
     |████████████████████████████████| 890kB 45.2MB/s 
     |████████████████████████████████| 1.1MB 43.4MB/s 
     |████████████████████████████████| 3.0MB 38.8MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893257 sha256=8ed954af1ace0fe7cc3ace367a81e05ee8e009d0ee0657da64c7bdd29cf963d1
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


In [5]:
!pip install contractions

     |████████████████████████████████| 245kB 7.1MB/s 
     |████████████████████████████████| 317kB 8.4MB/s 
  Created wheel for pyahocorasick: filename=pyahocorasick-1.4.0-cp36-cp36m-linux_x86_64.whl size=81700 sha256=b4765e18e638d262dbe9aca9f83ea2dfbe50ba9ea5ad6a463d08c772cf67375c
  Stored in directory: /root/.cache/pip/wheels/0a/90/61/87a55f5b459792fbb2b7ba6b31721b06ff5cf6bde541b40994
Successfully built pyahocorasick


In [6]:
import warnings
warnings.simplefilter('ignore')
import numpy as np
import pandas as pd
from tqdm import tqdm
from sklearn import metrics
import transformers
import torch
from torch.utils.data import Dataset, DataLoader, RandomSampler, SequentialSampler
from transformers import BertTokenizer, BertModel
import logging
from tqdm import tqdm
logging.basicConfig(level=logging.ERROR)

In [7]:
torch.__version__

'1.7.0+cu101'

In [8]:
from torch import cuda
device = 'cuda' if cuda.is_available() else 'cpu'


In [9]:
# test using cpu
device = 'cpu'

In [10]:
from sklearn.metrics import f1_score, precision_score, recall_score

In [11]:
!lscpu

Architecture:        x86_64
CPU op-mode(s):      32-bit, 64-bit
Byte Order:          Little Endian
CPU(s):              2
On-line CPU(s) list: 0,1
Thread(s) per core:  2
Core(s) per socket:  1
Socket(s):           1
NUMA node(s):        1
Vendor ID:           GenuineIntel
CPU family:          6
Model:               79
Model name:          Intel(R) Xeon(R) CPU @ 2.20GHz
Stepping:            0
CPU MHz:             2200.126
BogoMIPS:            4400.25
Hypervisor vendor:   KVM
Virtualization type: full
L1d cache:           32K
L1i cache:           32K
L2 cache:            256K
L3 cache:            56320K
NUMA node0 CPU(s):   0,1
Flags:               fpu vme de pse tsc msr pae mce cx8 apic sep mtrr pge mca cmov pat pse36 clflush mmx fxsr sse sse2 ss ht syscall nx pdpe1gb rdtscp lm constant_tsc rep_good nopl xtopology nonstop_tsc cpuid tsc_known_freq pni pclmulqdq ssse3 fma cx16 pcid sse4_1 sse4_2 x2apic movbe popcnt aes xsave avx f16c rdrand hypervisor lahf_lm abm 3dnowprefetch invpcid_sin

# **load and preprocess the dataframe**

In [12]:
df = pd.read_csv('/content/drive/MyDrive/isear_processed_emotions.csv')

#df = pd.read_csv('isear_processed_emotions.csv')

In [13]:
df = pd.get_dummies(df, columns=['emotions'])

In [14]:
df['labels'] = df[['emotions_anger', 'emotions_fear', 'emotions_joy', 'emotions_sadness']].values.astype('int').tolist()

In [15]:
df.shape

(7666, 9)

In [16]:
df = df[(df['emotions_anger']==1) | (df['emotions_fear']==1) | (df['emotions_joy']==1) | (df['emotions_sadness']==1)]

In [17]:
len(df)

4381

In [18]:
import re
import contractions
import unicodedata
import string

In [19]:
def clean_text(text):
    """
    Function to clean text with basic steps - lower casing, dealing with contractions, remove html codes,
    strip whitespaces, social media cleaning (remove hashtags and URLS), remove punctuationns, using regular expressions.
 
    Parameters
    ----------
    text : str
            Text to be cleaned
    
    Returns
    -------
    text : str
            Cleaned text
    """
    # Lower casing
    text = text.lower()
    
    
    # Remove html codes
    text = re.sub(r"&amp;", " ", text)
    text = re.sub(r"&quot;", " ", text)
    text = re.sub(r"&#39;", " ", text)
    text = re.sub(r"&gt;", " ", text)
    text = re.sub(r"&lt;", " ", text)
    
    # Strips (removes) whitespaces
    text = text.strip(' ')
    
    ################ Social media cleaning ############
    
    # Remove hashtags (Regex @[A-Za-z0-9]+ represents mentions and #[A-Za-z0-9]+ represents hashtags. )
    text = re.sub("(@[A-Za-z0-9]+)|(#[A-Za-z0-9]+)", " ", text)
    
    # Remove URLS (Regex \w+:\/\/\S+ matches all the URLs starting with http:// or https:// and replacing it with space.)
    text = re.sub("(\w+:\/\/\S+)", " ", text)
    text = re.sub(r'http\S+', ' ', text)
    
     # remove old style retweet text "RT"
    text = re.sub(r'^RT[\s]+', '', text)
    # remove accents
    text = unicodedata.normalize('NFKD', text).encode('ascii', 'ignore').decode('utf-8', 'ignore')
    # remove @users
    text = re.sub(r'@[\w]*', '', text)
    # remove Reddit channel reference /r
    text = re.sub(r'r/', '', text)
    
    # remove reddit username
    text = re.sub(r'u/[\w]*', '', text)
    # remove '&gt;' like notations
    text = re.sub('&\W*\w*\W*;', ' ', text)
    # remove hashtags
    text = re.sub(r'#[\w]*', '', text)
    ###################################################
    
    # Dealing with contractions
    text = contractions.fix(text)
    
    text = re.sub(r"what\'s", "what is ", text)
    text = re.sub(r"\'s", " ", text)
    text = re.sub(r"\'ve", " have ", text)
    text = re.sub(r"can\'t", "can not ", text)
    text = re.sub(r"n\'t", " not ", text)
    text = re.sub(r"\'t", " not", text )
    text = re.sub(r"i\'m", "i am ", text)
    text = re.sub(r"\'em'", " them ", text)
    text = re.sub(r"\'re", " are ", text)
    text = re.sub(r"\'d", " would ", text)
    text = re.sub(r"\'ll", " will ", text)
    text = re.sub(r"\'scuse", " excuse ", text)
    
    
    # Removes punctuations
    text = re.sub('['+string.punctuation+']', " ", text)
    
	# Removes non alphanumeric characters
    #text = re.sub('\W', ' ', text)
    
    # Removes non alphabetical characters
    text = re.sub('[^a-zA-Z]+', ' ', text)
    
    # Replaces all whitespaces by 1 whitespace
    text = re.sub('\s+', ' ', text)
    
    return text

In [20]:
df['text'] = df['text'].apply(clean_text)

In [21]:
df = df[["text", "labels"]]

In [22]:
df.reset_index(drop=True, inplace=True)

In [23]:
df.head()

,text,labels
0,during the period of falling in love each time...,"[0, 0, 1, 0]"
1,when i was involved in a traffic accident,"[0, 1, 0, 0]"
2,when i was driving home after several days of ...,"[1, 0, 0, 0]"
3,when i lost the person who meant the most to me,"[0, 0, 0, 1]"
4,when i got a letter offering me the summer job...,"[0, 0, 1, 0]"


# **load model**

In [24]:
class TinyBERTClass(torch.nn.Module):
    def __init__(self):
        super(TinyBERTClass, self).__init__()
        self.l1 = BertModel.from_pretrained('nreimers/TinyBERT_L-4_H-312_v2')
        self.dropout = torch.nn.Dropout(0)
        self.classifier = torch.nn.Linear(312, 4)
    def forward(self, input_ids, attention_mask, token_type_ids):
        output_1 = self.l1(input_ids=input_ids, attention_mask=attention_mask, token_type_ids=token_type_ids)
#         hidden_state = output_1[0]
#         pooler = hidden_state[:, 0]
        pooled_output = output_1[1]
        pooled_output = self.dropout(pooled_output)
        output = self.classifier(pooled_output)

        return output

In [25]:
predictor = torch.load('/content/drive/MyDrive/tinybert_emotions_epoch30_advait.bin', map_location=torch.device('cpu'))

***quantize the model***

# **make tensors and predict**

In [26]:
tokenizer = BertTokenizer.from_pretrained('nreimers/TinyBERT_L-4_H-312_v2')

# **using dataloader instead of loop**

In [27]:
class InferenceDataset(Dataset):
    def __init__(self, texts, tokenizer, max_len, padding=True):
        self.tokenizer = tokenizer
        self.text = texts
        self.max_len = max_len
        self.padding = padding

    def __len__(self):
        return len(self.text)

    def __getitem__(self, index):
        text = self.text[index]

        inputs = self.tokenizer.encode_plus(
            text,
            None,
            add_special_tokens=True,
            max_length=self.max_len,
            pad_to_max_length=self.padding,
            return_token_type_ids=True
        )
        ids = inputs['input_ids']
        mask = inputs['attention_mask']
        token_type_ids = inputs["token_type_ids"]


        return {
            'ids': torch.tensor(ids, dtype=torch.long),
            'mask': torch.tensor(mask, dtype=torch.long),
            'token_type_ids': torch.tensor(token_type_ids, dtype=torch.long)
        }

In [28]:
def predict_batch(predictor, input_text, tokenizer, MAX_LEN=64, batch_size=128, device='cpu', padding=True, quantize = True):
  """Function to run batch inference on input text"""

  if quantize:
    predictor = torch.quantization.quantize_dynamic(predictor, {torch.nn.Linear}, dtype=torch.qint8)

  data_set = InferenceDataset(input_text, tokenizer, MAX_LEN, padding)

  data_params = {'batch_size': batch_size,
                  'shuffle': False,
                  'num_workers': 0
                  }
  data_loader = DataLoader(data_set, **data_params)

  
  predictor.eval()
  pred_dataloader=[]
  
  with torch.no_grad():
      for _, data in tqdm(enumerate(data_loader, 0)):
          ids = data['ids'].to(device, dtype = torch.long)
          mask = data['mask'].to(device, dtype = torch.long)
          token_type_ids = data['token_type_ids'].to(device, dtype = torch.long)
          outputs = predictor(ids, mask, token_type_ids)
          pred_dataloader.extend(torch.sigmoid(outputs).cpu().detach().numpy().tolist())
  return pred_dataloader

In [29]:
def eval_isear(predy, df_labels):
  pred = np.array(predy)
  #.squeeze(1)
  for i in range(len(pred)):
    for j in range(4):
      pred[i, j] =1 if pred[i, j] >= 0.5 else 0
  pred = pred.astype('int')

  labels = df_labels.values.tolist()
  labels = np.array(labels)
  labels = labels.astype('int')

  precision = [precision_score(labels[:, 0], pred[:, 0]),
             precision_score(labels[:, 1], pred[:, 1]),
             precision_score(labels[:, 2], pred[:, 2]),
             precision_score(labels[:, 3], pred[:, 3]),
             precision_score(labels, pred, average='macro'),
             precision_score(labels, pred, average='micro'),
             precision_score(labels, pred, average='weighted')]

  recall = [recall_score(labels[:, 0], pred[:, 0]),
          recall_score(labels[:, 1], pred[:, 1]),
          recall_score(labels[:, 2], pred[:, 2]),
          recall_score(labels[:, 3], pred[:, 3]),
          recall_score(labels, pred, average='macro'),
          recall_score(labels, pred, average='micro'),
          recall_score(labels, pred, average='weighted')]

  f1 = [f1_score(labels[:, 0], pred[:, 0]),
      f1_score(labels[:, 1], pred[:, 1]),
      f1_score(labels[:, 2], pred[:, 2]),
      f1_score(labels[:, 3], pred[:, 3]),
      f1_score(labels, pred, average='macro'),
      f1_score(labels, pred, average='micro'),
      f1_score(labels, pred, average='weighted')]
  
  metric = pd.DataFrame([precision, recall, f1], columns=['anger', 'fear', 'joy', 'sadness', 'macro', 'micro', 'weighted'], index=['precision', 'recall','f1'])
  return metric

In [30]:
pred_y = predict_batch(predictor, df['text'], tokenizer, MAX_LEN=128, batch_size=1, device='cpu', padding = False, quantize = False)

4381it [00:54, 80.51it/s]


In [31]:
eval_isear(pred_y, df.labels)

,anger,fear,joy,sadness,macro,micro,weighted
precision,0.840069,0.972437,0.931436,0.848940,0.898220,0.890566,0.898188
recall,0.891423,0.805479,0.658135,0.876825,0.807966,0.808035,0.808035
f1,0.864985,0.881119,0.771291,0.862657,0.845013,0.847295,0.845038


# **export to onnx using previous method(version1: export a model which requires fixed length input)**

In [32]:
import torch.onnx

In [78]:
tokenizer = BertTokenizer.from_pretrained('nreimers/TinyBERT_L-4_H-312_v2', truncation=True, do_lower_case=True)

In [79]:
text = 'this is an challenging task'
input = tokenizer.encode_plus(
              text,
              None,
              add_special_tokens=True,
              max_length=128,
              pad_to_max_length=True,
              return_token_type_ids=True
          )
ids = torch.tensor([input['input_ids']], dtype=torch.long)
mask = torch.tensor([input['attention_mask']], dtype=torch.long)
token_type_ids = torch.tensor([input["token_type_ids"]], dtype=torch.long)

In [80]:
model = torch.nn.DataParallel(predictor)

In [ ]:
model.eval()

In [83]:
torch.onnx.export(
    model.module,
    (ids, mask, token_type_ids),
    '/content/drive/MyDrive/tinybert_emotions_detection.onnx',
    input_names=['ids', 'mask', 'ttids'],
    output_names= ['output'],
    dynamic_axes={
        'ids': {0: 'batch_size'}, 
        'mask': {0: 'batch_size'}, 
        'ttids': {0: 'batch_size'}, 
        'output': {0: 'batch_size'}
    }
)

# **load ONNX**

In [89]:
! pip install onnxruntime

In [90]:
import onnxruntime as ort

In [91]:
MODEL =  ort.InferenceSession('/content/drive/MyDrive/tinybert_emotions_detection.onnx')

In [92]:
tokenizer = BertTokenizer.from_pretrained('nreimers/TinyBERT_L-4_H-312_v2', truncation=True, do_lower_case=True)

In [93]:
def to_numpy(tensor):
  if tensor.requires_grad:
    return tensor.detach().cpu().numpy()
  else:
    return tensor.cpu().numpy()
import math
def sigmoid(x):
  return [1 / (1 + math.exp(-xi)) for xi in x]

In [94]:
data_set = InferenceDataset(df['text'], tokenizer, 128, True)

data_params = {'batch_size': 64,
                'shuffle': False,
                'num_workers': 0
                }
data_loader = DataLoader(data_set, **data_params)

In [95]:
pred_dataloader_onnx=[]
for _, data in tqdm(enumerate(data_loader, 0)):
    ids = to_numpy(data['ids'])
    mask = to_numpy(data['mask'])
    ttids = to_numpy(data['token_type_ids'])
    onnx_inputs = {'ids': ids, 'mask': mask, 'ttids':ttids}
    outputs = MODEL.run(None, onnx_inputs)
    for i in range(outputs[0].shape[0]):
        pred_dataloader_onnx.append(sigmoid(outputs[0][i]))



0it [00:00, ?it/s]

1it [00:01,  1.76s/it]

2it [00:03,  1.75s/it]

3it [00:05,  1.72s/it]

4it [00:06,  1.70s/it]

5it [00:08,  1.68s/it]

6it [00:10,  1.68s/it]

7it [00:11,  1.67s/it]

8it [00:13,  1.67s/it]

9it [00:15,  1.66s/it]

10it [00:16,  1.66s/it]

11it [00:18,  1.66s/it]

12it [00:20,  1.66s/it]

13it [00:21,  1.66s/it]

14it [00:23,  1.66s/it]

15it [00:24,  1.66s/it]

16it [00:26,  1.66s/it]

17it [00:28,  1.66s/it]

18it [00:29,  1.66s/it]

19it [00:31,  1.66s/it]

20it [00:33,  1.66s/it]

21it [00:34,  1.66s/it]

22it [00:36,  1.66s/it]

23it [00:38,  1.66s/it]

24it [00:39,  1.67s/it]

25it [00:41,  1.68s/it]

26it [00:43,  1.68s/it]

27it [00:45,  1.68s/it]

28it [00:46,  1.67s/it]

29it [00:48,  1.67s/it]

30it [00:50,  1.67s/it]

31it [00:51,  1.66s/it]

32it [00:53,  1.66s/it]

33it [00:54,  1.66s/it]

34it [00:56,  1.66s/it]

35it [00:58,  1.66s/it]

36it [00:59,  1.67s/it]

37it [01:01,  1.66s/it]

38it [01:03,  1.66s/it]

39it [01:04,  1.66s/it]

40it [01:06,

In [96]:
eval_isear(pred_dataloader_onnx, df.labels)

,anger,fear,joy,sadness,macro,micro,weighted
precision,0.840069,0.972437,0.931436,0.848940,0.898220,0.890566,0.898188
recall,0.891423,0.805479,0.658135,0.876825,0.807966,0.808035,0.808035
f1,0.864985,0.881119,0.771291,0.862657,0.845013,0.847295,0.845038


# **use convert method provided by torch to export onnx: maybe support dynamic input size**

In [113]:
mymodel = torch.load('/content/drive/MyDrive/tinybert_emotions_epoch30_advait.bin', map_location=torch.device('cpu'))

In [120]:
mymodel.save_pretraiend('tinybert')

ModuleAttributeError: ignored

In [111]:
from transformers.convert_graph_to_onnx import convert
from pathlib import Path
# Handles all the above steps for you
convert(framework="pt", model='model_folder', output=Path("content/drive/tinybert.onnx"), opset=11)

ONNX opset version set to: 11
Loading pipeline (model: TinyBertClass, tokenizer: TinyBertClass)


OSError: ignored

maybe upload the model to huggingface repo first